In [3]:
import ee
ee.Authenticate()  
ee.Initialize()

# Taking data

In [6]:
import geemap 

def process_indices(roi_bounds, start_date, end_date, drive_folder='LandslideData'):
    # Define ROI (geometry from bounds)
    roi = ee.Geometry.Rectangle(roi_bounds)  # e.g., [-122.0, 37.0, -121.0, 38.0]

    # Step 2.1: Fetch Landsat 8/9 Surface Reflectance (pre-atmospherically corrected, NASA/USGS data)
    landsat = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')  # Or LC09 for Landsat 9
               .filterDate(start_date, end_date)
               .filterBounds(roi)
               .filter(ee.Filter.lt('CLOUD_COVER', 20))  # Basic cloud filter
               .median())  # Median composite for cloud-free mosaic

    # Step 2.2: Cloud Masking (more precise)
    def mask_clouds(image):
        qa = image.select('QA_PIXEL')
        cloud_bit_mask = 1 << 3  # Cloud bit
        cirrus_bit_mask = 1 << 2  # Cirrus bit
        mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
        return image.updateMask(mask).divide(10000)  # Scale SR to 0-1 reflectance
    landsat = landsat.map(mask_clouds)

    # Select bands (Landsat 8: B2=blue, B3=green, B4=red, B5=NIR)
    optical_bands = landsat.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5'])

    # Step 2.3: Compute Indices
    # NDVI: (NIR - Red) / (NIR + Red)
    ndvi = optical_bands.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

    # NDWI: (Green - NIR) / (Green + NIR) for water/soil moisture
    ndwi = optical_bands.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')

    # Slope from SRTM DEM (NASA data, 30m res)
    dem = ee.Image('USGS/SRTMGL1_003').clip(roi)
    slope = ee.Terrain.slope(dem).rename('slope')  # In degrees

    # Step 2.4: Optional Change Detection (e.g., NDVI difference for "before/after")
    # Split dates if needed (e.g., before_end = '2025-01-15')
    # before = process up to before_end, after = rest, diff = after.subtract(before)

    # Step 2.5: Export to Google Drive (as separate tasks; monitor with ee.batch.Task.list())
    task_ndvi = ee.batch.Export.image.toDrive(
        image=ndvi.clip(roi),
        description=f'ndvi_{start_date.replace("-", "")}',
        folder=drive_folder,
        region=roi,
        scale=30,  # Landsat res
        crs='EPSG:4326',
        maxPixels=1e9,
        fileFormat='GeoTIFF'
    )
    task_ndvi.start()

    task_ndwi = ee.batch.Export.image.toDrive(
        image=ndwi.clip(roi),
        description=f'ndwi_{start_date.replace("-", "")}',
        folder=drive_folder,
        region=roi,
        scale=30,
        crs='EPSG:4326',
        maxPixels=1e9,
        fileFormat='GeoTIFF'
    )
    task_ndwi.start()

    task_slope = ee.batch.Export.image.toDrive(
        image=slope.clip(roi),
        description='slope_static',  # Slope is static, no date filter
        folder=drive_folder,
        region=roi,
        scale=30,
        crs='EPSG:4326',
        maxPixels=1e9,
        fileFormat='GeoTIFF'
    )
    task_slope.start()

    return {'tasks': [task_ndvi.id, task_ndwi.id, task_slope.id]}  # Return task IDs for monitoring